##### Dependencies
____

In [28]:
import numpy as np
import pandas as pd
import sqlalchemy as db
import datetime
import random
import hashlib

from faker import Faker

##### Database connection
____

In [2]:
g_params = {
    'dbuser'   : 'csc4710',
    'dbpwd'    : 'password',
    'dbserver' : 'localhost',
    'dbport'   : 3306,
    'dbname'   : 'CSC4710'
}

disk_engine = db.create_engine(f'mysql+pymysql://{g_params["dbuser"]}:{g_params["dbpwd"]}@{g_params["dbserver"]}:{g_params["dbport"]}/{g_params["dbname"]}', echo=False)
disk_engine.connect().connection.ping()

##### Load Credentials Dataframe
____

In [3]:
credentials = pd.read_sql_table(table_name='credentials', con=disk_engine)

credentials.head(5)

,userid,password,employee_id,driver_id,customer_id,role
0,1,ccd537fc55c7bfa0c64910999ae28ea12823439ce7394d...,1.0,NaN,NaN,employee
1,2,eed6735ba9b046d3a275e87124fbe8c3e49b5928b484cc...,2.0,NaN,NaN,employee
2,3,27c410d2a6124f27f16b5d0a521249be3eec7ab24d5f91...,3.0,NaN,NaN,employee
3,4,855fb422ee1f140faf1837005435858554b90bf2521a1a...,4.0,NaN,NaN,employee
4,5,9a432d3d6b2bf4b422a6e9b22f5e49668ef621236aa882...,5.0,NaN,NaN,employee


##### Add Payment Method
____

In [30]:
faker = Faker(locale='en_US')

for ind in credentials.index:    
    
    userid = credentials['userid'][ind].__int__()
    
    payment_method = random.randint(1, 3)
    
    if (payment_method == 1):
        credit_card_number = hashlib.sha256(faker.credit_card_number().encode()).hexdigest()        
        branch             = faker.credit_card_provider()
        expiration_month   = faker.credit_card_expire().split('/')[0]
        expiration_year    = faker.credit_card_expire().split('/')[1]
        
        sql = "INSERT INTO credit_card(credit_card_id, branch, expiration_month, expiration_year) VALUES (%s, %s, %s, %s)"
        values = (credit_card_number, branch, expiration_month, expiration_year)
    
        disk_engine.execute(sql, values)
        
        sql = "INSERT INTO payment_method(userid, credit_card_id) VALUES (%s, %s)"
        values = (userid, credit_card_number)
        
        disk_engine.execute(sql, values)
        
    elif (payment_method == 2):
        comments = 'Autogenerated'
        uid     = faker.sha256()
        
        sql = "INSERT INTO paypal(comments, uid) VALUES (%s, %s)"
        values = (comments, uid)
    
        paypal_id = disk_engine.execute(sql, values).lastrowid
        
        sql = "INSERT INTO payment_method(userid, paypal_id) VALUES (%s, %s)"
        values = (userid, paypal_id)
        
        disk_engine.execute(sql, values)
        
    else:
        comments = 'Autogenerated'
        uid     = faker.sha256()
        
        sql = "INSERT INTO google_pay(comments, uid) VALUES (%s, %s)"
        values = (comments, uid)
    
        google_id = disk_engine.execute(sql, values).lastrowid
        
        sql = "INSERT INTO payment_method(userid, google_pay_id) VALUES (%s, %s)"
        values = (userid, google_id)
        
        disk_engine.execute(sql, values)        
